In [136]:
import pandas as pd

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv")
#df.dropna()

COLUMN_NA_THRESHOLD = 0.2 # if at least 20% of column is null, drop column.
OHE_NUNIQUE_VALS = 10 # if number of unique values in column is less than this, apply OneHotEncoder.

#print(df.isnull().mean())
cleaned_df = df.loc[:, df.isnull().mean() < COLUMN_NA_THRESHOLD] # 'age' column is almost 20% brah... this is bad.. maybe  -- IterativeImputer !     one issue with this : Categorical rows to be OneHotEncoded before Imputation: now that we've done this threshold, we can basically dropna ? For example, Embarked contains 2 null values - so just drop both rows. Issue comes when there is more - we don't wanna drop more rows (imagine if Age was a Categorical instead...) 
removed_columns = df.columns.difference(cleaned_df.columns) # ^^^ We have to OneHotEncode Categorical columns in order to Impute dataframe. What if these columns contain large number of null values? ---- also look at 

# ^ basically the problem is, can not impute categorical data.
# for GA, optimise IterativeImputer ?? (it can use forest etc in back)

# SEPARATE NUMERIC AND CATEGORICAL DATA
numeric_cols = cleaned_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = cleaned_df.select_dtypes(include=['object']).columns

# encode columns AFTER being imputed in order to avoid nan cols ..... encode BEFORE..? BUT: 
# dont impute columns with no null values ofc
# if categorical, check its in cols_to_encode - otherwise just ignore / drop the row ... too much hassle 
# columns like Name and Ticket won't give many details to the imputer anyway, so we can just ignore them 


cols_to_encode = [col for col in categorical_cols if df[col].nunique() < OHE_NUNIQUE_VALS]
print("cols_to_encode: ", cols_to_encode)

# dropna in cols_to_encode (i.e Embarked has 2 null values - we dont want to pass that onto OHE.)
encode_df = df[cols_to_encode].dropna()
df = df.loc[encode_df.index] # ensure rows dropped in encode_df are dropped in df aswell.

# some values in column 
encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
encoded_cats = encoder.fit_transform(encode_df)


# Convert encoded features into a DataFrame
encoded_cat_df = pd.DataFrame(encoded_cats, 
                               columns=encoder.get_feature_names_out(cols_to_encode),
                               index=encode_df.index)

print("numeric_cols: ", numeric_cols)
print("categorical_cols: ", categorical_cols)
print("cols_to_encode: ", cols_to_encode)
print("ImputeIgnore", categorical_cols.difference(cols_to_encode))


# CONCATENATE NUMERIC + ENCODED CATEGORICAL DATA
to_be_imputed = pd.concat([df[numeric_cols], encoded_cat_df], axis=1)


# IMPUTE MISSING VALUES WITH ITERATIVE IMPUTER (Using RandomForest)
imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100, random_state=42), 
                           max_iter=1000, random_state=42)


imputed_df = imputer.fit_transform(to_be_imputed)

# after imputed, concat ImputeIgnore values to it and should be good



cols_to_encode:  ['Sex', 'Embarked']
numeric_cols:  Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
categorical_cols:  Index(['Name', 'Sex', 'Ticket', 'Embarked'], dtype='object')
cols_to_encode:  ['Sex', 'Embarked']
ImputeIgnore Index(['Name', 'Ticket'], dtype='object')


TypeError: concat() takes 1 positional argument but 2 were given

In [58]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

cleaned_df

# Convert categorical features to numeric using one-hot encoding
#pd.get_dummies(cleaned_df, columns=['Sex', 'Embarked'], drop_first=True)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C
